In [1]:
video_path = "../demo/demo-3s-small.mp4"

# Use ffmpeg to decompose
import os
# Create directories if they don't exist
if not os.path.exists("frames"):
    os.makedirs("frames")
if not os.path.exists("decomposed_frames"):
    os.makedirs("decomposed_frames")

os.system(f"ffmpeg -i {video_path} frames/output_%04d.png")

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

0

In [2]:
import sys
sys.path.append(os.path.abspath("../"))
from scripts.decompose_image import decompose_image 
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("frames", i), os.path.join("decomposed_frames", i[:-4]))

Saved R channel to decomposed_frames/output_0001_R.txt
Saved G channel to decomposed_frames/output_0001_G.txt
Saved B channel to decomposed_frames/output_0001_B.txt
Saved R channel to decomposed_frames/output_0002_R.txt
Saved G channel to decomposed_frames/output_0002_G.txt
Saved B channel to decomposed_frames/output_0002_B.txt
Saved R channel to decomposed_frames/output_0003_R.txt
Saved G channel to decomposed_frames/output_0003_G.txt
Saved B channel to decomposed_frames/output_0003_B.txt
Saved R channel to decomposed_frames/output_0004_R.txt
Saved G channel to decomposed_frames/output_0004_G.txt
Saved B channel to decomposed_frames/output_0004_B.txt
Saved R channel to decomposed_frames/output_0005_R.txt
Saved G channel to decomposed_frames/output_0005_G.txt
Saved B channel to decomposed_frames/output_0005_B.txt
Saved R channel to decomposed_frames/output_0006_R.txt
Saved G channel to decomposed_frames/output_0006_G.txt
Saved B channel to decomposed_frames/output_0006_B.txt
Saved R ch

In [2]:
import os
import shutil
import re

def copy_trimmed_files(a, b, output_dir):
    input_dir = 'decomposed_frames'

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Pattern to match filenames like output_0001_R.txt
    pattern = re.compile(r'output_(\d{4})_[RGB]\.txt')

    for filename in os.listdir(input_dir):
        match = pattern.match(filename)
        if match:
            frame_num = int(match.group(1))
            if a <= frame_num <= b:
                src = os.path.join(input_dir, filename)
                dst = os.path.join(output_dir, filename)
                shutil.copyfile(src, dst)
                print(f"Copied {filename}")

copy_trimmed_files(10,40,'trimmed_decomposed_frames')
copy_trimmed_files(10,10,'trimmed_decomposed_frames_2')

Copied output_0010_G.txt
Copied output_0010_R.txt
Copied output_0010_B.txt


In [4]:
import os
import numpy as np
from PIL import Image
import re

def recompose_images():
    input_dir = 'trimmed_decomposed_frames'
    output_dir = 'trimmed_frames'
    os.makedirs(output_dir, exist_ok=True)

    # Pattern to extract NNNN and channel
    pattern = re.compile(r'output_(\d{4})_([RGB])\.txt')

    # Collect files by frame number
    frame_dict = {}
    for filename in os.listdir(input_dir):
        match = pattern.match(filename)
        if match:
            frame_num, channel = match.groups()
            frame_dict.setdefault(frame_num, {})[channel] = filename

    # Reconstruct images
    for frame_num, channels in frame_dict.items():
        if all(c in channels for c in 'RGB'):
            r = np.loadtxt(os.path.join(input_dir, channels['R']), dtype=np.uint8)
            g = np.loadtxt(os.path.join(input_dir, channels['G']), dtype=np.uint8)
            b = np.loadtxt(os.path.join(input_dir, channels['B']), dtype=np.uint8)

            # Stack into a 3D array
            img_array = np.stack([r, g, b], axis=2)

            # Create and save the image
            img = Image.fromarray(img_array, 'RGB')
            output_path = os.path.join(output_dir, f'output_{frame_num}.png')
            img.save(output_path)
            print(f"Saved image {output_path}")
        else:
            print(f"Incomplete channels for frame {frame_num}, skipping.")

# Example usage
recompose_images()


Saved image trimmed_frames/output_0012.png
Saved image trimmed_frames/output_0010.png
Saved image trimmed_frames/output_0030.png
Saved image trimmed_frames/output_0014.png
Saved image trimmed_frames/output_0036.png
Saved image trimmed_frames/output_0034.png
Saved image trimmed_frames/output_0029.png
Saved image trimmed_frames/output_0016.png
Saved image trimmed_frames/output_0032.png
Saved image trimmed_frames/output_0017.png
Saved image trimmed_frames/output_0037.png
Saved image trimmed_frames/output_0035.png
Saved image trimmed_frames/output_0013.png
Saved image trimmed_frames/output_0011.png
Saved image trimmed_frames/output_0033.png
Saved image trimmed_frames/output_0031.png
Saved image trimmed_frames/output_0028.png
Saved image trimmed_frames/output_0015.png
Saved image trimmed_frames/output_0022.png
Saved image trimmed_frames/output_0026.png
Saved image trimmed_frames/output_0024.png
Saved image trimmed_frames/output_0019.png
Saved image trimmed_frames/output_0039.png
Saved image

In [10]:
import os
import shutil

input_dir = "trimmed_frames"
temp_dir = "temp_numbered_frames"
output_video = "reconstructed_video.mp4"

# Create temp dir
os.makedirs(temp_dir, exist_ok=True)

# Collect and sort PNGs by frame number
png_files = sorted(
    [f for f in os.listdir(input_dir) if f.endswith(".png")],
    key=lambda x: int(x.split("_")[1].split(".")[0])
)

# Copy them as 0001.png, 0002.png, etc.
for i, filename in enumerate(png_files):
    src = os.path.join(input_dir, filename)
    dst = os.path.join(temp_dir, f"{i+1:04d}.png")
    shutil.copyfile(src, dst)

# Run ffmpeg with image sequence
os.system(f"ffmpeg -y -framerate 15 -i {temp_dir}/%04d.png -c:v libx264 -pix_fmt yuv420p {output_video}")

# Clean up (optional)
# shutil.rmtree(temp_dir)

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

0